In [1]:
import sys 
print(sys.executable)

/Users/danielching/miniforge3/envs/selenium_env/bin/python


In [2]:
%conda install pandas

/Users/danielching/miniforge3/lib/python3.10/site-packages/conda_package_streaming/package_streaming.py:25: UserWarning: zstandard could not be imported. Running without .conda support.
  warnings.warn("zstandard could not be imported. Running without .conda support.")
/Users/danielching/miniforge3/lib/python3.10/site-packages/conda_package_handling/api.py:29: UserWarning: Install zstandard Python bindings for .conda support
  _warnings.warn("Install zstandard Python bindings for .conda support")
Retrieving notices: ...working... done
Channels:
 - conda-forge
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import requests

# Indiv Activities Processing 

In [10]:
import os
import pandas as pd
import re

def detect_wrongly_tagged_paces(input_folder):
    """
    Detect and flag `_indiv_activities` CSV files with unrealistic paces based on headers.

    Args:
        input_folder (str): Path to the folder containing `_indiv_activities` files.
    
    Returns:
        None
    """
    flagged_files = []

    # Loop through all `_indiv_activities` files
    for filename in os.listdir(input_folder):
        if '_indiv_activities' in filename and filename.endswith('.csv'):
            file_path = os.path.join(input_folder, filename)
            try:
                # Load the file
                data = pd.read_csv(file_path)

                # Check if the header contains "min/mi"
                if 'Pace (min/mi)' in data.columns:
                    # Group by Athlete ID to calculate average pace for each athlete
                    athlete_groups = data.groupby(['Athlete ID', 'Athlete Name'])

                    for (athlete_id, athlete_name), group in athlete_groups:
                        avg_pace = group['Pace (min/mi)'].mean()

                        # Flag if the average pace is in the 4.xx range
                        if 4.00 <= avg_pace < 5.00:
                            flagged_files.append({
                                'File': filename,
                                'Athlete ID': athlete_id,
                                'Athlete Name': athlete_name,
                                'Average Pace (min/mi)': avg_pace
                            })
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

    # Print flagged files and athletes
    if flagged_files:
        print("Potentially wrongly tagged files and athletes:")
        for entry in flagged_files:
            print(f"File: {entry['File']}, Athlete: {entry['Athlete Name']} (ID: {entry['Athlete ID']}), "
                  f"Avg Pace: {entry['Average Pace (min/mi)']:.2f}")
    else:
        print("No wrongly tagged files or athletes detected.")

# Example usage


In [7]:
detect_wrongly_tagged_paces(input_folder='./data')


Potentially wrongly tagged files and athletes:
File: batch_45_indiv_activities.csv, Athlete: Sondre Nordstad Moen (ID: 14983457), Avg Pace: 4.98
File: batch_39_indiv_activities.csv, Athlete: Niels Laros (ID: 29900318), Avg Pace: 4.53
File: batch_46_indiv_activities.csv, Athlete: Kirami Collin Yego (ID: 73487204), Avg Pace: 4.61
File: batch_34_indiv_activities.csv, Athlete: Haftom Welday (ID: 23836673), Avg Pace: 4.75


In [11]:
import os
import pandas as pd

def merge_individual_activities_by_metric(input_folder, output_min_mi_file, output_min_km_file):
    """
    Merge `_indiv_activities` CSV files into two separate files based on pace metrics.
    
    Args:
        input_folder (str): Path to the folder containing `_indiv_activities` files.
        output_min_mi_file (str): Path to save the merged data with `min/mi` metric.
        output_min_km_file (str): Path to save the merged data with `min/km` metric.
    
    Returns:
        None
    """
    merged_min_mi = pd.DataFrame()
    merged_min_km = pd.DataFrame()
    files_processed = 0
    skipped_files = 0

    for filename in os.listdir(input_folder):
        if '_indiv_activities' in filename and filename.endswith('.csv'):
            file_path = os.path.join(input_folder, filename)
            try:
                # Read the current file
                current_data = pd.read_csv(file_path)

                # Check which metric the file contains
                if 'Pace (min/mi)' in current_data.columns:
                    # Merge with `min/mi` DataFrame
                    merged_min_mi = pd.concat([merged_min_mi, current_data], ignore_index=True)
                    print(f"Processed file with 'min/mi': {filename}")
                elif 'Pace (min/km)' in current_data.columns:
                    # Merge with `min/km` DataFrame
                    merged_min_km = pd.concat([merged_min_km, current_data], ignore_index=True)
                    print(f"Processed file with 'min/km': {filename}")
                else:
                    # Skip files with inconsistent or missing headers
                    print(f"Skipped file due to missing pace metric: {filename}")
                    skipped_files += 1
                    continue

                files_processed += 1
            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                skipped_files += 1

    # Save the merged files
    if not merged_min_mi.empty:
        merged_min_mi.to_csv(output_min_mi_file, index=False)
        print(f"Merged data with 'min/mi' saved to {output_min_mi_file}")
    else:
        print(f"No files with 'min/mi' metric were processed.")

    if not merged_min_km.empty:
        merged_min_km.to_csv(output_min_km_file, index=False)
        print(f"Merged data with 'min/km' saved to {output_min_km_file}")
    else:
        print(f"No files with 'min/km' metric were processed.")

    print(f"\nTotal files processed: {files_processed}, Skipped files: {skipped_files}")



In [13]:
# Example usage
merge_individual_activities_by_metric(
    input_folder='./data',
    output_min_mi_file='./data/merged_min_mi.csv',
    output_min_km_file='./data/merged_min_km.csv'
)


Processed file with 'min/mi': batch_18_indiv_activities.csv
Processed file with 'min/mi': batch_9_indiv_activities.csv
Processed file with 'min/mi': batch_20_indiv_activities.csv
Processed file with 'min/mi': batch_25_indiv_activities.csv
Processed file with 'min/mi': batch_53_indiv_activities.csv
Processed file with 'min/mi': batch_12_indiv_activities.csv
Processed file with 'min/mi': batch_17_indiv_activities.csv
Processed file with 'min/mi': batch_6_indiv_activities.csv
Processed file with 'min/mi': batch_44_indiv_activities.csv
Processed file with 'min/mi': batch_41_indiv_activities.csv
Processed file with 'min/mi': batch_38_indiv_activities.csv
Processed file with 'min/mi': batch_37_indiv_activities.csv
Processed file with 'min/mi': batch_32_indiv_activities.csv
Processed file with 'min/mi': batch_13_indiv_activities.csv
Processed file with 'min/mi': batch_2_indiv_activities.csv
Processed file with 'min/mi': batch_52_indiv_activities.csv
Processed file with 'min/mi': batch_7_indiv

In [22]:
import pandas as pd

def convert_min_mi_to_min_km(input_file, output_file):
    """
    Convert `Pace (min/mi)` in the input file to `Pace (min/km)` and save the output.
    Handles time formats like `1h 14m`, `50m 1s`, and `0m XXs`.

    Args:
        input_file (str): Path to the input CSV file with `min/mi` metrics.
        output_file (str): Path to save the converted CSV file.

    Returns:
        None
    """
    # Load data
    data = pd.read_csv(input_file)

    # Ensure numeric columns are correct
    data['Distance (mi)'] = pd.to_numeric(data['Distance (mi)'], errors='coerce').fillna(0).astype(float)
    data['Pace (min/mi)'] = pd.to_numeric(data['Pace (min/mi)'], errors='coerce').fillna(0).astype(float)

    # Convert Pace (min/mi) to Pace (min/km)
    data['Pace (min/km)'] = data['Pace (min/mi)'] / 1.60934

    # Convert Time to decimal minutes
    def time_to_minutes(time_str):
        try:
            hours, minutes, seconds = 0, 0, 0
            
            # Parse "1h 14m" format
            if "h" in time_str:
                parts = time_str.split("h")
                hours = int(parts[0].strip())
                time_str = parts[1].strip()
            
            # Parse "50m 1s" or "0m XXs"
            if "m" in time_str:
                parts = time_str.split("m")
                minutes = int(parts[0].strip())
                if "s" in parts[1]:
                    seconds = int(parts[1].replace("s", "").strip())
            elif "s" in time_str:
                # Handle "0m XXs" or "XXs"
                seconds = int(time_str.replace("s", "").strip())

            # Convert to decimal minutes
            return round(hours * 60 + minutes + seconds / 60, 2)
        except:
            return 0  # Default for invalid entries

    data['Time (min)'] = data['Time'].apply(time_to_minutes)

    # Remove old Time column (optional)
    data.drop(columns=['Time'], inplace=True)

    # Rename Distance column to `Distance (km)` for consistency
    data['Distance (km)'] = data['Distance (mi)'] * 1.60934
    data.drop(columns=['Distance (mi)'], inplace=True)

    # Save the converted data
    data.to_csv(output_file, index=False)
    print(f"Converted file saved to {output_file}")



In [23]:
# Example usage
convert_min_mi_to_min_km(
    input_file='./data/merged_min_mi.csv',
    output_file='./data/converted_min_km.csv'
)


Converted file saved to ./data/converted_min_km.csv


In [24]:
def convert_time_in_min_km(input_file, output_file):
    """
    Convert `Time` in the input file with `min/km` metrics into decimal minutes.

    Args:
        input_file (str): Path to the input CSV file with `min/km` metrics.
        output_file (str): Path to save the converted CSV file.

    Returns:
        None
    """
    # Load data
    data = pd.read_csv(input_file)

    # Ensure numeric columns are correct
    data['Distance (km)'] = pd.to_numeric(data['Distance (km)'], errors='coerce').fillna(0).astype(float)
    data['Pace (min/km)'] = pd.to_numeric(data['Pace (min/km)'], errors='coerce').fillna(0).astype(float)

    # Convert Time to decimal minutes
    def time_to_minutes(time_str):
        try:
            hours, minutes, seconds = 0, 0, 0
            
            # Parse "1h 14m" format
            if "h" in time_str:
                parts = time_str.split("h")
                hours = int(parts[0].strip())
                time_str = parts[1].strip()
            
            # Parse "50m 1s" or "0m XXs"
            if "m" in time_str:
                parts = time_str.split("m")
                minutes = int(parts[0].strip())
                if "s" in parts[1]:
                    seconds = int(parts[1].replace("s", "").strip())
            elif "s" in time_str:
                # Handle "0m XXs" or "XXs"
                seconds = int(time_str.replace("s", "").strip())

            # Convert to decimal minutes
            return round(hours * 60 + minutes + seconds / 60, 2)
        except:
            return 0  # Default for invalid entries

    data['Time (min)'] = data['Time'].apply(time_to_minutes)

    # Remove old Time column (optional)
    data.drop(columns=['Time'], inplace=True)

    # Save the converted data
    data.to_csv(output_file, index=False)
    print(f"Converted file saved to {output_file}")



In [25]:

# Example usage
convert_time_in_min_km(
    input_file='./data/merged_min_km.csv',
    output_file='./data/converted_min_km_with_time.csv'
)

Converted file saved to ./data/converted_min_km_with_time.csv


In [26]:
def merge_and_add_activity_time(file1, file2, output_file):
    """
    Merge two converted files and add `Activity Time (s)` column.

    Args:
        file1 (str): Path to the first input CSV file.
        file2 (str): Path to the second input CSV file.
        output_file (str): Path to save the merged CSV file.

    Returns:
        None
    """
    # Load both datasets
    data1 = pd.read_csv(file1)
    data2 = pd.read_csv(file2)

    # Merge datasets
    merged_data = pd.concat([data1, data2], ignore_index=True)

    # Ensure `Time (min)` is numeric
    merged_data['Time (min)'] = pd.to_numeric(merged_data['Time (min)'], errors='coerce').fillna(0)

    # Add `Activity Time (s)` as an integer column
    merged_data['Activity Time (s)'] = (merged_data['Time (min)'] * 60).astype(int)

    # Save the merged dataset
    merged_data.to_csv(output_file, index=False)
    print(f"Merged file saved to {output_file}")

# Example usage
merge_and_add_activity_time(
    file1='./data/converted_min_km.csv',
    file2='./data/converted_min_km_with_time.csv',
    output_file='./data/indiv_activities_full.csv'
)


Merged file saved to ./data/indiv_activities_full.csv


In [32]:
import pandas as pd
import numpy as np

def clean_athlete_name(name: str) -> str:
    return " ".join(name.strip().title().split())

def load_and_clean_data():
    activities = pd.read_csv("./data/indiv_activities_full.csv")
    # Convert to datetime and ensure no timezone issues
    activities['Start Date'] = pd.to_datetime(activities['Start Date'], utc=True)
    activities['Start Date'] = activities['Start Date'].dt.tz_convert(None)

    male_iaaf = pd.read_csv("combined_df_male_distanceCAA30Oct (1).csv")
    female_iaaf = pd.read_csv("combined_df_female_distanceCAA30Oct.csv")

    male_iaaf['Gender'] = 'Male'
    female_iaaf['Gender'] = 'Female'
    iaaf = pd.concat([male_iaaf, female_iaaf], ignore_index=True)

    iaaf['Competitor'] = iaaf['Competitor'].apply(clean_athlete_name)
    activities['Athlete Name'] = activities['Athlete Name'].apply(clean_athlete_name)

    return activities, iaaf


def filter_activities_by_date(activities: pd.DataFrame, start_date='2024-01-01', weeks=45):
    end_date = pd.to_datetime(start_date) + pd.Timedelta(weeks=weeks)
    filtered = activities[(activities['Start Date'] >= start_date) & (activities['Start Date'] < end_date)]
    return filtered

def prepare_activity_types(activities: pd.DataFrame):
    # Normalize activity types
    # Combine VirtualRide and Ride into Ride
    activities['Type'] = activities['Type'].replace('VirtualRide', 'Ride')
    # For future-proofing, if we have Swim and Run, they remain as is.
    # Any type not in [Run, Ride, Swim] becomes Other
    valid_types = ['Run', 'Ride', 'Swim']
    activities['Type'] = activities['Type'].apply(lambda x: x if x in valid_types else 'Other')
    return activities

def compute_activity_hours(activities: pd.DataFrame):
    # If Activity Time (s) not available or NA, use Elapsed Time
    # Assume Elapsed Time is in seconds (based on the given snippet).
    # If Activity Time (s) is NaN, fallback to Elapsed Time.
    activities['Activity Time (s)'] = activities['Activity Time (s)'].fillna(activities['Elapsed Time'])
    # Convert to hours
    activities['Activity Hours'] = activities['Activity Time (s)'] / 3600.0
    return activities

def aggregate_training_metrics(activities: pd.DataFrame):
    # Calculate total and average (per 45 weeks) metrics
    grouped = activities.groupby(['Athlete Name', 'Type'], as_index=False).agg({
        'Distance (km)': 'sum',
        'Activity Hours': 'sum'
    })

    pivot_dist = grouped.pivot(index='Athlete Name', columns='Type', values='Distance (km)').fillna(0)
    pivot_time = grouped.pivot(index='Athlete Name', columns='Type', values='Activity Hours').fillna(0)

    # Run metrics
    total_run_distance = pivot_dist['Run'] if 'Run' in pivot_dist.columns else pd.Series(0, index=pivot_dist.index)
    total_run_hours = pivot_time['Run'] if 'Run' in pivot_time.columns else pd.Series(0, index=pivot_dist.index)

    avg_weekly_run_mileage = total_run_distance / 45.0
    avg_weekly_run_hours = total_run_hours / 45.0

    metrics_df = pd.DataFrame({
        'Athlete Name': pivot_dist.index,
        'Total_Run_Distance_km': total_run_distance,
        'Avg_Weekly_Run_Mileage_km': avg_weekly_run_mileage,
        'Total_Run_Hours': total_run_hours,
        'Avg_Weekly_Run_Hours': avg_weekly_run_hours
    }).reset_index(drop=True)

    # Add columns for other activities
    # We assume columns might be 'Ride', 'Swim', 'Other'
    for activity_type in ['Ride', 'Swim', 'Other']:
        if activity_type in pivot_time.columns:
            metrics_df[f'Total_{activity_type}_Hours'] = pivot_time[activity_type].values
        else:
            metrics_df[f'Total_{activity_type}_Hours'] = 0.0

    return metrics_df

def merge_iaaf_data(metrics_df: pd.DataFrame, iaaf: pd.DataFrame):
    # Aggregate IAAF data
    iaaf_agg = iaaf.groupby('Competitor', as_index=False).agg(
    Nat=('Nat', 'first'),
    Gender=('Gender', 'first'),
    Mark=('Mark', lambda x: '|'.join(x.dropna().unique())),
    Discipline=('Discipline', lambda x: '|'.join(x.dropna().unique())),
    Number_of_events=('Competitor', 'count')
    )


    merged = metrics_df.merge(iaaf_agg, left_on='Athlete Name', right_on='Competitor', how='left')

    # Fill missing values
    merged['Nat'] = merged['Nat'].fillna('')
    merged['Gender'] = merged['Gender'].fillna('')
    merged['Mark'] = merged['Mark'].fillna('')
    merged['Discipline'] = merged['Discipline'].fillna('')
    merged['Number_of_events'] = merged['Number_of_events'].fillna(0).astype(int)

    return merged

def check_for_na_and_alert(df: pd.DataFrame):
    # Check if any NA present
    if df.isna().any().any():
        na_cols = df.columns[df.isna().any()].tolist()
        print(f"Warning: The following columns contain NA values: {na_cols}")
    else:
        print("No NA values found in the final DataFrame.")


activities, iaaf = load_and_clean_data()
filtered_activities = filter_activities_by_date(activities)
filtered_activities = prepare_activity_types(filtered_activities)
filtered_activities = compute_activity_hours(filtered_activities)
metrics_df = aggregate_training_metrics(filtered_activities)
final_df = merge_iaaf_data(metrics_df, iaaf)




/var/folders/6t/bxcnwybj72j7sr4fzj25fj3c0000gn/T/ipykernel_36834/4158597234.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities['Type'] = activities['Type'].replace('VirtualRide', 'Ride')
/var/folders/6t/bxcnwybj72j7sr4fzj25fj3c0000gn/T/ipykernel_36834/4158597234.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  activities['Type'] = activities['Type'].apply(lambda x: x if x in valid_types else 'Other')
/var/folders/6t/bxcnwybj72j7sr4fzj25fj3c0000gn/T/ipykernel_36834/4158597234.py:45: SettingW

In [33]:
check_for_na_and_alert(final_df)


In [ ]:
final_df.to_csv("cleaned_athlete_metadata.csv", index=False)

In [34]:
def add_average_run_pace(final_df: pd.DataFrame) -> pd.DataFrame:
    # Ensure no division by zero
    # Run pace in min/km = (Total_Run_Hours * 60) / Total_Run_Distance_km
    # If distance is zero, set pace to NaN or a default value
    final_df['Avg_Run_Pace_min_per_km'] = (final_df['Total_Run_Hours'] * 60) / final_df['Total_Run_Distance_km']
    final_df.loc[final_df['Total_Run_Distance_km'] == 0, 'Avg_Run_Pace_min_per_km'] = np.nan
    
    return final_df


In [36]:
final_df = add_average_run_pace(final_df)
final_df.to_csv("cleaned_athlete_metadata.csv", index=False)


# Metadata Processing 

In [ ]:
import os
import pandas as pd
import re

def merge_meta_files_with_batch_debug(input_folder, output_file):
    merged_data = pd.DataFrame()
    files_processed = 0
    skipped_files = 0
    consistent_headers = None

    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)

        # Skip non-CSV files or files containing "_indiv_activities" in the name
        if not filename.endswith('.csv') or '_indiv_activities' in filename:
            continue

        try:
            # Read the current file
            current_data = pd.read_csv(file_path)

            # Extract the batch number from the filename using regex
            batch_match = re.search(r'batch_(\d+)', filename)
            if batch_match:
                batch_number = batch_match.group(1)
            else:
                batch_number = "Unknown"
                print(f"Warning: Could not extract batch number from filename {filename}")

            # Add a "Batch" column to track the batch source
            current_data['Batch'] = batch_number
            print(f"Added 'Batch' column with value '{batch_number}' to file: {filename}")

            # Check and enforce consistent headers
            if consistent_headers is None:
                consistent_headers = set(current_data.columns)
                merged_data = current_data
            elif set(current_data.columns) == consistent_headers:
                merged_data = pd.concat([merged_data, current_data], ignore_index=True)
            else:
                skipped_files += 1
                print(f"Skipped file due to inconsistent headers: {filename}")
                continue

            files_processed += 1
            print(f"Processed: {filename}")

        except Exception as e:
            print(f"Error processing file {filename}: {e}")
            skipped_files += 1

    # Save the merged data to the output file
    if not merged_data.empty:
        merged_data.to_csv(output_file, index=False)
        print(f"\nMerging complete! Processed {files_processed} files, skipped {skipped_files} files.")
        print(f"Merged data saved to {output_file}.")
    else:
        print("\nNo files were merged. Check the input folder and headers.")


In [ ]:
merge_meta_files_with_batch_debug(input_folder='./data', output_file='./data/full_meta.csv')


In [ ]:

def process_full_meta(file_path, output_file):
    # Load the merged data
    data = pd.read_csv(file_path)

    # Ensure 'Batch' column is structured as integers
    if 'Batch' in data.columns:
        data['Batch'] = pd.to_numeric(data['Batch'], errors='coerce').astype('Int64')

    # Ensure 'Distance (km)' column is structured as floats
    if 'Distance (km)' in data.columns:
        data['Distance (km)'] = pd.to_numeric(data['Distance (km)'], errors='coerce')

    # Find duplicate names across different batch numbers
    if 'Name' in data.columns and 'Batch' in data.columns:
        duplicate_names = (
            data.groupby('Name')['Batch']
            .nunique()
            .reset_index()
            .query('Batch > 1')
            ['Name']
        )
        
        # Filter out entries with the later batch number for these names
        duplicates_filtered = data[data['Name'].isin(duplicate_names)]
        data = data[~data['Name'].isin(duplicate_names)]
        
        earliest_batches = duplicates_filtered.groupby('Name')['Batch'].min().reset_index()
        duplicates_filtered = duplicates_filtered.merge(earliest_batches, on=['Name', 'Batch'], how='inner')
        data = pd.concat([data, duplicates_filtered], ignore_index=True)

        print(f"Filtered out duplicate names across batches: {list(duplicate_names)}")

    # Save the processed data
    data.to_csv(output_file, index=False)
    print(f"Processed data saved to {output_file}.")



In [ ]:
process_full_meta(file_path='./data/full_meta.csv', output_file='./data/processed_full_meta.csv')


In [ ]:
import pandas as pd
import numpy as np

def process_and_sort_athletes(input_file, output_sorted_csv, output_statistics_csv):
    # Load data
    data = pd.read_csv(input_file)

    # Ensure Athlete ID is treated as integers
    data['Athlete ID'] = pd.to_numeric(data['Athlete ID'], errors='coerce').fillna(0).astype(int)

    # Ensure Distance (km) column is numeric
    data['Distance (km)'] = pd.to_numeric(data['Distance (km)'], errors='coerce').fillna(0)

    # Group by Athlete ID to calculate consistency metrics
    athlete_groups = data.groupby('Athlete ID')

    # Create a DataFrame for athlete statistics
    athlete_stats = []
    for athlete_id, group in athlete_groups:
        total_weeks = len(group)
        valid_weeks = (group['Distance (km)'] > 0).sum()
        consistency_percentage = (valid_weeks / total_weeks) * 100 if total_weeks > 0 else 0
        std_dev = group['Distance (km)'].std() if valid_weeks > 0 else float('inf')
        median_distance = group['Distance (km)'].median()
        mean_distance = group['Distance (km)'].mean()
        max_distance = group['Distance (km)'].max()
        median_to_mean_ratio = median_distance / mean_distance if mean_distance > 0 else 0
        max_to_median_ratio = max_distance / median_distance if median_distance > 0 else float('inf')
        no_data_count = (group['Distance (km)'] == 0).sum()
        
        athlete_name = group['Name'].iloc[0]  
        athlete_stats.append({
            'Athlete ID': athlete_id,
            'Athlete Name': athlete_name,  
            'Consistency Percentage': consistency_percentage,
            'Standard Deviation': std_dev,
            'Median-to-Mean Ratio': median_to_mean_ratio,
            'Max-to-Median Ratio': max_to_median_ratio,
            'No Data Count': no_data_count
        })

    # Create DataFrame of statistics
    athlete_stats_df = pd.DataFrame(athlete_stats)

    # Sort by Consistency Percentage, then by Standard Deviation, and push "No Data" athletes to the bottom
    athlete_stats_df.sort_values(
        by=['Consistency Percentage', 'Standard Deviation', 'No Data Count'], 
        ascending=[False, True, True], 
        inplace=True
    )

    # Create a categorical sorting order based on sorted Athlete ID
    sorting_order = pd.Categorical(data['Athlete ID'], categories=athlete_stats_df['Athlete ID'], ordered=True)
    sorted_data = data.sort_values(by='Athlete ID', key=lambda x: sorting_order)

    # Save the sorted data and statistics
    sorted_data.to_csv(output_sorted_csv, index=False)
    athlete_stats_df.to_csv(output_statistics_csv, index=False)

    print(f"Sorted original CSV saved to {output_sorted_csv}")
    print(f"Athlete statistics CSV saved to {output_statistics_csv}")



In [ ]:

# Example usage
process_and_sort_athletes(
    input_file='./data/processed_full_meta.csv',
    output_sorted_csv='./data/sorted_full_meta.csv',
    output_statistics_csv='./data/athlete_statistics.csv'
)